In [1]:
import PyPDF2
from PyPDF2 import PdfReader
import re
import pandas as pd
import tabula
import numpy as np 
import warnings
import time
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
file = pd.read_csv("Turn Data/Turn Data.csv").Filepath.squeeze() # get file path

pdf_file = open(file, 'rb') # Open the PDF file
pdf_reader = PyPDF2.PdfReader(pdf_file) # Create a PDF reader object

xlist = []
def analyse(i):
    
    reader = PdfReader(file)
    
    page = reader.pages
        # top, left, bottom, right
    global nwo
    
    nwo = pd.DataFrame(tabula.read_pdf(file, pages=i)[0]) # extract data from the table
    
    #xlist.append(nwo)

# --------- strip special characters ---------- #
    nwo.iloc[:,3:16] = nwo.iloc[:,3:16].astype(str).map(lambda x: x.rstrip('*+-')) # 
    nwo.drop(list(nwo.filter(regex='Unnamed: 0')), axis=1, inplace=True)
    nwo["SA"] = nwo["SA"].fillna("")
     # --------- break into groups ---------- #
    s = nwo.eq(nwo.columns)
    s_groups = [g.reset_index(drop=True) for _, g in nwo[~s.iloc[:, 0]].groupby(s.cumsum()[~s.iloc[:, 0]].iloc[:, 0])]
    s_gks = s_groups[0].drop(index=0)
    s_deff = s_groups[1].drop(index=0)
    s_mid = s_groups[2].drop(index=0)
    s_att = s_groups[3].drop(index=0)

    # --------- rename cols ---------- #
    s_deff = s_deff.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud','Crs':'Vis'})
    s_mid = s_mid.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud','Crs':'Vis'})
    s_att = s_att.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud','Crs':'Vis'})

    # --------- split ID and names ---------- #
    s_gks[['ID', 'Name']] = s_gks['ID Name'].str.split(' ', n=1, expand=True)
    s_gks.drop(columns="ID Name",axis=1, inplace=True)
    s_gks = s_gks[["ID", "Name", "Age", "Foot", "Sho", "Mov","Pas","Con","Spe","Sta","Hea","Str","Dis","Han","Ref","Crs","OA","SA"]]


    
    # --------- Calculate PV ---------- #
    s_gks['GK'] = s_gks[['Dis', 'Han', 'Ref','Crs']].astype(int).sum(axis = 1, skipna = True)
    
    s_deff['CB'] = s_deff[['Hea', 'Str', 'Tac','Jud']].astype(int).sum(axis = 1, skipna = True)
    s_deff['FB'] = s_deff[['Spe', 'Sta', 'Agg','Tac']].astype(int).sum(axis = 1, skipna = True)
    s_deff['SW'] = s_deff[['Pas', 'Con', 'Tac','Jud']].astype(int).sum(axis = 1, skipna = True)
    s_deff['WB'] = s_deff[['Mov', 'Pas', 'Spe','Sta']].astype(int).sum(axis = 1, skipna = True)

    s_mid['CM'] = s_mid[['Pas', 'Sta', 'Hea','Tac']].astype(int).sum(axis = 1, skipna = True)
    s_mid['AM'] = s_mid[['Str', 'Agg', 'Tac','Jud']].astype(int).sum(axis = 1, skipna = True)
    s_mid['PL'] = s_mid[['Pas', 'Con', 'Jud','Vis']].astype(int).sum(axis = 1, skipna = True)
    s_mid['FR'] = s_mid[['Sho', 'Mov', 'Pas','Con']].astype(int).sum(axis = 1, skipna = True)
    s_mid['WG'] = s_mid[['Pas', 'Con', 'Spe','Sta']].astype(int).sum(axis = 1, skipna = True)

    s_att['WF'] = s_att[['Sho', 'Mov', 'Con','Spe']].astype(int).sum(axis = 1, skipna = True)
    s_att['IF'] = s_att[['Pas', 'Con', 'Jud','Vis']].astype(int).sum(axis = 1, skipna = True)
    s_att['CF'] = s_att[['Sho', 'Mov', 'Str','Agg']].astype(int).sum(axis = 1, skipna = True)
    s_att['TM'] = s_att[['Con', 'Hea', 'Str','Agg']].astype(int).sum(axis = 1, skipna = True)

    s_PV=29
    # --------- don't show low PVs ---------- #
    s_gks['GK'] = s_gks['GK'].astype(int); s_gks['GK'] = np.where(s_gks['GK'] < s_PV, "", s_gks['GK'])

    s_deff['CB'] = s_deff['CB'].astype(int); s_deff['CB'] = np.where(s_deff['CB'] < s_PV, "", s_deff['CB'])
    s_deff['SW'] = s_deff['SW'].astype(int); s_deff['SW'] = np.where(s_deff['SW'] < s_PV, "", s_deff['SW'])
    s_deff['WB'] = s_deff['WB'].astype(int); s_deff['WB'] = np.where(s_deff['WB'] < s_PV, "", s_deff['WB'])
    s_deff['FB'] = s_deff['FB'].astype(int); s_deff['FB'] = np.where(s_deff['FB'] < s_PV, "", s_deff['FB'])
    
    s_mid['CM'] = s_mid['CM'].astype(int); s_mid['CM'] = np.where(s_mid['CM'] < s_PV, "", s_mid['CM'])
    s_mid['AM'] = np.where(s_mid['AM'] < s_PV, "", s_mid['AM'])
    s_mid['PL'] = np.where(s_mid['PL'] < s_PV, "", s_mid['PL'])
    s_mid['FR'] = np.where(s_mid['FR'] < s_PV, "", s_mid['FR'])
    s_mid['WG'] = s_mid['WG'].astype(int); s_mid['WG'] = np.where(s_mid['WG'] < s_PV, "", s_mid['WG'])

    s_att['WF'] = s_att['WF'].astype(int); s_att['WF'] = np.where(s_att['WF'] < s_PV, "", s_att['WF'])
    s_att['IF'] = np.where(s_att['IF'] < s_PV, "", s_att['IF'])
    s_att['TM'] = np.where(s_att['TM'] < s_PV, "", s_att['TM'])
    s_att['CF'] = np.where(s_att['CF'] < s_PV, "", s_att['CF'])


    # --------- Show Players by Section ---------- #
    display("Goalkeepers",s_gks.style.background_gradient(axis=None, vmax=10,vmin=1, subset=['Spe','Dis','Crs']),
            "Defenders",s_deff.style.background_gradient(axis=None, vmax=10,vmin=1, subset=['Spe','Agg','Vis']),
            "Midfielders",s_mid.style.background_gradient(axis=None, vmax=10,vmin=1, subset=['Spe','Agg','Vis']),
            "Attackers",s_att.style.background_gradient(axis=None, vmax=10,vmin=1, subset=['Spe','Agg','Vis'])
           )
    
    
opp = []
opp.clear()
search_string = " Opponents - " # set desired searchable text 

for page_number in range(len(pdf_reader.pages)): # search file for text and append each page where this is found to the list
        text = pdf_reader.pages[page_number].extract_text() 
        if search_string in text: 
            pg = page_number + 1
            opp.append(pg)

print(opp)

[45, 46, 47]


In [3]:
# send to analyse function
i = 0
while i < len(opp):
    result = analyse(opp[i])
    i+=1


'Goalkeepers'

,ID,Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Dis,Han,Ref,Crs,OA,SA,GK
1,POLJ,J Pollersbeck,26,R,9,9,8,8,6,8,8,7,10,9,9,9,100,Pre/Hei,37
2,ROJL,L Rojas,22,R,8,5,6,7,6,9,7,8,8,9,9,9,91,,35


'Defenders'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Min Price,Max Price,CB,FB,SW,WB
1,RUIM M Rui,30,L,9,9,10,10,9,10,10,8,8,8,8,7,106,Mid/Exp,"£42,397,418","£75,799,281",34,35,36,38
2,ROSS S Rosted,26,R,9,8,9,8,9,9,9,9,8,10,10,6,104,Cha/Det,"£91,000,000","£91,000,000",38,36,37,35
3,PMPF L Page,25,R,9,7,8,7,8,10,10,9,9,9,9,7,102,Rel/Loy,"£31,805,790","£56,253,018",37,36,33,33
4,ANDJ J Anderson,24,B,9,10,9,8,9,10,6,7,9,8,9,6,100,Mid/Det,"£39,129,059","£70,867,165",30,36,34,38
5,RTPE L Rodríguez,28,L,8,9,9,8,9,10,4,10,7,10,8,8,100,Gam,"£14,700,119","£28,978,729",32,36,35,37
6,SAAL L Saatka,25,R,10,9,9,8,7,9,8,8,9,10,9,4,100,Cha/Rel,"£20,711,520","£38,095,219",35,35,36,34
7,CRWK G Cau,24,L,6,8,9,7,9,10,7,7,7,9,9,9,97,Mid,"£14,820,949","£27,446,202",32,35,34,36
8,HAWE E Hawke,23,L,8,6,7,6,7,10,10,9,9,9,9,6,96,,"£16,114,927","£29,842,458",37,35,31,30


'Midfielders'

...

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Unnamed: 1,Price,CB,FB,SW,WB
1,CDWW R Convery,21,L,5,9,6,8,10,10,10,9,6,10,6,7,96,Inx,"£14,153,865","£26,210,860",35,36,30,35
2,KUGF B Koskinen,21,R,7,9,10,8,9,10,7,7,7,8,6,8,96,Inx/Pro,"£20,835,202","£35,983,331",,34,32,38
3,NPMA Nagore,19,R,8,10,8,8,10,9,6,7,8,7,7,6,94,,"£13,979,030","£25,887,092",,34,30,37
4,RHRP A Rankin,19,R,7,6,9,6,7,9,9,9,8,10,7,7,94,Mid,"£15,688,090","£29,052,019",35,34,32,31
5,JOAE E Joao Batista,18,L,8,8,9,8,10,6,6,7,7,8,4,9,90,,"£2,651,225","£4,909,677",,31,29,33
6,CDVE N Chiquinho,19,R,7,7,7,8,10,7,7,9,7,8,4,6,87,Inx,"£1,300,000","£2,066,194",,32,,31
7,GKHW S Groves,17,R,4,3,5,5,6,4,6,7,7,5,3,4,59,Inx,£0,"£6,355",,,,


'Midfielders'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Unnamed: 1,Price,CM,AM,PL,FR,WG
1,BAXW N Burgos,19,R,7,4,9,8,8,7,7,10,8,9,7,10,94,Inx,"£6,209,636","£11,499,327",32,34,34,,32
2,WNPX D Wallace (Inj),19,R,6,7,6,9,6,9,10,7,8,9,6,5,88,Inx/Loy,"£1,484,928","£5,194,193",34,30,,,30
3,SZUF F Sutter,18,R,4,5,5,4,9,5,3,7,9,9,6,10,76,Fla/Inx,"£50,000","£183,015",,31,,,
4,AUMD F Appleton,16,R,4,5,5,4,7,5,6,6,8,8,8,6,72,Inx/Loy,£0,"£7,424",,30,,,
5,OAKO O Oakley,19,L,3,5,5,4,7,4,5,6,8,3,3,6,59,Inx,£0,"£7,187",,,,,
6,BUNQ U Boland,20,R,4,5,3,4,6,4,6,7,5,2,4,6,56,Inx,£0,"£7,142",,,,,


'Attackers'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Unnamed: 1,Price,WF,IF,CF,TM
1,LMNB N Livermore,20,B,8,9,7,9,9,9,7,9,6,9,6,6,94,Inx,"£9,198,998","£17,035,181",35,,32,31
2,VDRS A Velasco,20,R,9,8,9,8,10,10,7,6,6,7,6,6,92,Inx,"£7,382,249","£13,670,832",35,29,29,
3,HKHS S Harris (Inj),21,L,9,8,9,8,7,9,8,8,6,6,4,4,86,Inx,"£435,581","£1,958,961",32,,31,30
4,SDNU S Stringer,19,L,3,5,4,4,3,3,4,4,7,4,4,6,51,Inx,£0,"£7,814",,,,
5,SETW E Sedgley,18,R,2,3,3,4,7,4,6,7,4,3,4,3,50,Inx,£0,"£7,569",,,,
